<a href="https://colab.research.google.com/github/dasari2023/Code/blob/main/VGGNetIDRiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import image_data_format
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.models import Sequential
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from keras.layers import Dense
from keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D
from keras.layers import AvgPool2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
#from keras.constraints import maxnorm
import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import sys
import glob
import keras
import cv2
import csv
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint


In [ ]:
pip install pytesseract


In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import image_to_string

In [ ]:
pip install opencv-python


In [ ]:
x,y1, y = [], [],[]
#train_labels = {
   # "fire":0,"nofire":1
#}
x_train=list()
y_train=list()
#fire = glob.glob('/content/sample_data/Forest Fire Dataset/Training/fire/*.jpg')
#nfire =glob.glob('/content/sample_data/Forest Fire Dataset/Training/nofire/*.jpg')
dgtrain =glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Training/*.jpg')
ltrain =glob.glob('/content/drive/MyDrive/Dataset/Localization/Training/*.jpg')
strain =glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Training/*.jpg')
for i in dgtrain:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(0)
for j in ltrain:
    img=cv2.imread(j,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(1)
for k in strain:
    img=cv2.imread(k,1)
    img=cv2.resize(img,(224,224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(2)

In [ ]:
#firel = glob.glob('/content/sample_data/Forest Fire Dataset/Testing/fire/*.jpg')
#nfirel=glob.glob('/content/sample_data/Forest Fire Dataset/Testing/nofire/*.jpg')
dgtest = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Testing/*.jpg')
#dgtest =glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Training/*.jpg')
ltest =glob.glob('/content/drive/MyDrive/Dataset/Localization/Testing/*.jpg')
stest =glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Testing/*.jpg')
x_test=list()
y_test=list()
for i in dgtest:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(0)
for j in ltest:
    img=cv2.imread(j,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(1)
for k in stest:
    img=cv2.imread(k,1)
    img=cv2.resize(img,(224, 224))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(224, 224, 3)
    x.append(img)
    y.append(2)

In [ ]:

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your dataset paths
dgtrain = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Training/*.jpg')
ltrain = glob.glob('/content/drive/MyDrive/Dataset/Localization/Training/*.jpg')
strain = glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Training/*.jpg')

dgtest = glob.glob('/content/drive/MyDrive/Dataset/Disease Grading/Testing/*.jpg')
ltest = glob.glob('/content/drive/MyDrive/Dataset/Localization/Testing/*.jpg')
stest = glob.glob('/content/drive/MyDrive/Dataset/Segmentation/Testing/*.jpg')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path, 1)
    img = cv2.resize(img, (224, 224))
    img = np.float32(img) / 255.0
    img = img.reshape(224, 224, 3)
    return img

# Load and preprocess training data
x = []
y = []

for i in dgtrain:
    img = load_and_preprocess_image(i)
    x.append(img)
    y.append(0)

for j in ltrain:
    img = load_and_preprocess_image(j)
    x.append(img)
    y.append(1)

for k in strain:
    img = load_and_preprocess_image(k)
    x.append(img)
    y.append(2)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Create an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate augmented images and append to the training set
for i in range(len(x_train)):
    img = x_train[i]
    img = img.reshape((1,) + img.shape)  # Reshape to (1, height, width, channels) for flow method
    for batch in datagen.flow(img, batch_size=1):
        x_train.append(batch[0])
        y_train.append(y_train[i])
        break  # Exit the loop after one batch to avoid infinite loop

# Similarly, you can perform data augmentation for validation data if needed

# Convert lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

# Now, x_train and y_train contain augmented training data
# x_val and y_val contain validation data
# You can use these arrays to train your model





In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have loaded your data into the variables x and y correctly
# x should contain the images, and y should contain the labels.

# Convert data to NumPy arrays
x_samp = np.asarray(x)
y_samp = np.asarray(y)

# Check the shapes of your data to ensure they match
print("x_samp.shape:", x_samp.shape)
print("y_samp.shape:", y_samp.shape)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_samp, y_samp, test_size=0.2, random_state=42)

# Check the shapes of the split datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
#print("x_test shape:", x_test.shape)
#print("y_test shape:", y_test.shape)

x_samp.shape: (880, 224, 224, 3)
y_samp.shape: (880,)
x_train shape: (704, 224, 224, 3)
y_train shape: (704,)


In [ ]:
pip install keras scikit-learn

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Assuming x and y are your data and labels
x_samp = np.asarray(x)
y_samp = np.asarray(y)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_samp, y_samp, test_size=0.2, random_state=42)

# Use one-hot encoding for the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# VGGNet model with pre-trained weights (you can modify this based on your needs)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()

# Add the VGGNet base model
model.add(base_model)

# Flatten the output of VGGNet
model.add(layers.Flatten())

# Add some dense layers for classification
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # Assuming binary classification, adjust for your needs

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation to improve generalization
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(x_train, y_train_encoded, batch_size=32),
    steps_per_epoch=len(x_train) // 32,
    epochs=10,
    validation_data=(x_test, y_test_encoded),
    verbose=1
)

58889256/58889256 [==============================] - 1s 0us/step


Epoch 1/10
22/22 [==============================] - 631s 29s/step - loss: 4.6420 - accuracy: 0.4517 - val_loss: 2.0655 - val_accuracy: 0.4886
Epoch 2/10
22/22 [==============================] - 624s 29s/step - loss: 1.8105 - accuracy: 0.4688 - val_loss: 0.8523 - val_accuracy: 0.4886
Epoch 3/10
22/22 [==============================] - 625s 29s/step - loss: 0.9544 - accuracy: 0.4659 - val_loss: 0.7032 - val_accuracy: 0.4886
Epoch 4/10
22/22 [==============================] - 620s 28s/step - loss: 0.7165 - accuracy: 0.4602 - val_loss: 0.6736 - val_accuracy: 0.4886
Epoch 5/10
22/22 [==============================] - 618s 28s/step - loss: 0.6908 - accuracy: 0.4631 - val_loss: 0.6769 - val_accuracy: 0.4886
Epoch 6/10
22/22 [==============================] - 618s 28s/step - loss: 0.6775 - accuracy: 0.4616 - val_loss: 0.6744 - val_accuracy: 0.4886
Epoch 7/10
22/22 [==============================] - 599s 27s/step - loss: 0.6830 - accuracy: 0.4688 - val_loss: 0.6751 - val_accuracy: 0.4886
Epoch 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import numpy as np
import copy
import random
import csv
x = np.array(x)
y=np.array(y)
# Client config
NUM_OF_CLIENTS = 3
SELECT_CLIENTS = 0.5
EPOCHS = 4
CLIENT_EPOCHS = 5
BATCH_SIZE = 10
DROP_RATE = 0

# Define serverhist dictionary to store evaluation metrics
serverhist = {
    'loss': [],
    'accuracy': [],
    'f1': [],
    'precision': [],
    'recall': [],
    'specificity': [],
    'sensitivity': []
}

# Corrected code
evaluation = [7]
result = evaluation[0]
#print(result)

# Model config
LOSS = 'categorical_crossentropy'
NUM_OF_CLASSES = 3
learning_rate = 0.0025

# VGGNet model with pre-trained weights (you can modify this based on your needs)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model
model = Sequential()

# Add the VGGNet base model
model.add(base_model)

# Flatten the output of VGGNet
model.add(layers.Flatten())

# Add some dense layers for classification
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # Assuming binary classification, adjust for your needs

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# Split the data for each client
x_train_clients = np.array_split(x_train, NUM_OF_CLIENTS)
y_train_clients = np.array_split(y_train, NUM_OF_CLIENTS)
y_train_clients_one_hot = [to_categorical(labels) for labels in y_train_clients]


y_test_categorical = to_categorical(y_test)
# Use x_test directly for evaluation
y_test_one_hot = to_categorical(y_test, num_classes=3)

def init_model():
    fl_model = Sequential()
    fl_model.add(base_model)
    fl_model.add(Flatten())
    fl_model.add(Dense(256, activation='relu'))
    fl_model.add(Dense(NUM_OF_CLASSES, activation='softmax'))
    fl_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return fl_model

def client_update(index, client, now_epoch, avg_weight):
    #print("Client {}/{} fitting".format(index + 1, int(NUM_OF_CLIENTS * SELECT_CLIENTS)))
    client_model = client_models[index]
    if now_epoch != 0:
        client.set_weights(avg_weight)

    client.fit(x_train_clients[index], to_categorical(y_train_clients[index], num_classes=3),
               epochs=CLIENT_EPOCHS,
               batch_size=BATCH_SIZE,
               verbose=1,
               validation_split=0.2,
               )

    return client


# Define fedAVG function
def fedAVG(server_weight):
    avg_weight = np.array(server_weight[0])

    if len(server_weight) > 1:
        for i in range(1, len(server_weight)):
            avg_weight += server_weight[i]

        avg_weight = avg_weight / len(server_weight)

    return avg_weight

# Initialization
fl_model = init_model()
fl_model.summary()

# Split the data for each client
x_train_clients = np.array_split(x_train, NUM_OF_CLIENTS)
y_train_clients = np.array_split(y_train, NUM_OF_CLIENTS)
y_train_clients_one_hot = [to_categorical(labels) for labels in y_train_clients]

# Initialize avg_weight
avg_weight = fl_model.get_weights()

# Federated Learning Loop
for epoch in range(EPOCHS):
    client_models = [init_model() for _ in range(NUM_OF_CLIENTS)]

    server_weight = []
    selected_num = int(max(NUM_OF_CLIENTS * SELECT_CLIENTS, 1))
    split_data_index = random.sample(range(NUM_OF_CLIENTS), selected_num)

    for index in split_data_index:
        client_models[index] = client_update(index, client_models[index], epoch, avg_weight)
        server_weight.append(copy.deepcopy(client_models[index].get_weights()))

    avg_weight = fedAVG(server_weight)
    fl_model.set_weights(avg_weight)

    #print("Server {}/{} evaluate".format(epoch + 1, EPOCHS))
    # Use x_test directly for evaluation
    y_test_int = np.array(y_test)  # Convert to numpy array if not already
    evaluation = model.evaluate(x_test, y_test_categorical, batch_size=10, verbose=1)
    #print("Test loss:", evaluation[0])
    #print("Test accuracy:", evaluation[1])
    #print("Test f1:", evaluation[2])
    #print("Test precision:", evaluation[3])
    #print("Test recall:", evaluation[4])
    #print("Test specificity:", evaluation[5])
    #print("Test sensitivity:", evaluation[6])
    serverhist['accuracy'].append(evaluation[0])
    serverhist['loss'].append(evaluation[0])
    serverhist['f1'].append(evaluation[0])
    serverhist['precision'].append(evaluation[0])
    serverhist['recall'].append(evaluation[0])
    serverhist['specificity'].append(evaluation[0])
    serverhist['sensitivity'].append(evaluation[0])


# Print the results
print("\t\t\tFed Avg")
print("Round\tAccuracy\tLoss\tf1\tprecision\trecall\tspecificity\tsensitivity")
for i in range(EPOCHS):
    print("{}\t{}\t{}".format(
        i + 1,
        serverhist['accuracy'][i],
        serverhist['loss'][i],
        serverhist['f1'][i],
        serverhist['precision'][i],
        serverhist['recall'][i],
        serverhist['specificity'][i],
        serverhist['sensitivity'][i]
    ))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


20/20 [==============================] - 522s 26s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 2/10
20/20 [==============================] - 513s 26s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 3/10
20/20 [==============================] - 503s 25s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 4/10
20/20 [==============================] - 502s 25s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 5/10
20/20 [==============================] - 505s 25s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 6/10
20/20 [==============================] - 504s 25s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_loss: 0.0000e+00 - val_accuracy: 0.4789
Epoch 7/10
20/20 [==============================] - 467s 23s/step - loss: 0.0000e+00 - accuracy: 0.4724 - val_l

<ipython-input-14-868fb5efe013>:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  avg_weight = np.array(server_weight[0])


ValueError: ignored